In [1]:
from fastai.basics import *
from fastai.vision.all import *
from fastai.callback.all import *
from fastai.distributed import *
from fastprogress import fastprogress
from torchvision.models import *
from fastai.vision.models.xresnet import *
from fastai.callback.mixup import *

torch.backends.cudnn.benchmark = True
fastprogress.MAX_COLS = 80

I combine hyperparameter changes (`RandomResizedCrop` -> `Resize`, smaller `bs`) from [the notebook introducing mini-net](https://github.com/pete88b/data-science/blob/master/fastai-things/train-imagenette-mininet.ipynb) with [BlurPool](https://arxiv.org/pdf/1904.11486.pdf) [from this notebook](https://github.com/ducha-aiki/imagewoofv2-fastv2-maxpoolblur/blob/master/fastai2-imagenette-train-maxblurpool.ipynb) using the implementation from [kornia](https://github.com/kornia/kornia).

I train with `fp32` since my card does not support `fp16`.

The mean top 1 accuracy of 5 runs of training with new hyperaparmenters, using `xse_resnext50`, without `BlurPool`, is `0.864`.

**Mean of 5 runs with `BlurPool` is 0.875.**

### Random Erasing

https://arxiv.org/abs/1708.04896

* complementary to commonly used data augmentation
*  yields consistent improvement over strong baselines in image classification, object detection and person re-identification
* is lightweight (little extra computation, no extra memory), doesn't require involved hyperparam tuning
* improves the robustness of CNNs to partially occluded samples. From the paper: "When we randomly adding occlusion
to the CIFAR-10 testing dataset, Random Erasing significantly outperforms the baseline model"

### Mish

https://arxiv.org/abs/1908.08681

* works better
* slightly higher computational cost

### BlurPool

[Making Convolutional Networks Shift-Invariant Again](https://arxiv.org/abs/1904.11486)

In [2]:
def get_dataloaders(
    bs=64,
    item_tfms=[RandomResizedCrop(size=128, min_scale=0.35), FlipItem(0.5)],
    batch_tfms=RandomErasing(p=0.9, max_count=3)
):
    dblock = DataBlock(
        blocks=(ImageBlock, CategoryBlock),
        splitter=GrandparentSplitter(valid_name='val'),
        get_items=get_image_files,
        get_y=parent_label,
        item_tfms=item_tfms,
        batch_tfms=batch_tfms
    )

    return dblock.dataloaders(untar_data(URLs.IMAGENETTE_320), path=untar_data(URLs.IMAGENETTE_320), bs=bs, num_workers=8)

### Without BlurPool

In [3]:
for i in range(5):
    learn = Learner(
        get_dataloaders(bs=32, batch_tfms=[], item_tfms=[Resize(size=128), FlipItem(0.5)]),
        xse_resnext50(n_out=10, act_cls=Mish, sa=1, sym=0, pool=MaxPool),
        opt_func=partial(ranger, mom=0.95, sqr_mom=0.99, eps=1e-6, beta=0),
        metrics=[accuracy,top_k_accuracy],
        loss_func=LabelSmoothingCrossEntropy()
    )

    learn.fit_flat_cos(5, 8e-3, wd=1e-2, cbs=[])

epoch,train_loss,valid_loss,accuracy,top_k_accuracy,time
0,1.412588,1.401734,0.642038,0.940637,00:59
1,1.195296,1.149097,0.747006,0.966115,00:56
2,1.086665,1.232353,0.711592,0.964841,00:56
3,1.007356,1.168265,0.737070,0.969172,00:56
4,0.863072,0.873014,0.857325,0.988025,00:56


epoch,train_loss,valid_loss,accuracy,top_k_accuracy,time
0,1.433484,1.350375,0.654522,0.954140,00:56
1,1.199293,1.287421,0.693248,0.951847,00:56
2,1.080901,1.188214,0.731720,0.960510,00:56
3,1.030311,1.056093,0.797962,0.969172,00:56
4,0.861473,0.867688,0.865223,0.986752,00:56


epoch,train_loss,valid_loss,accuracy,top_k_accuracy,time
0,1.404817,1.521293,0.594140,0.897580,00:56
1,1.193953,1.229608,0.717197,0.963057,00:56
2,1.102215,1.117007,0.760510,0.972739,00:56
3,1.023358,1.262024,0.701656,0.957962,00:56
4,0.859508,0.869556,0.868280,0.986497,00:56


epoch,train_loss,valid_loss,accuracy,top_k_accuracy,time
0,1.374534,1.449893,0.629045,0.947516,00:56
1,1.228588,1.254478,0.690955,0.966879,00:56
2,1.087921,1.110399,0.761783,0.970955,00:56
3,1.030918,1.123557,0.755414,0.967643,00:56
4,0.874820,0.873644,0.865478,0.985223,00:56


epoch,train_loss,valid_loss,accuracy,top_k_accuracy,time
0,1.412081,1.328634,0.667006,0.950318,00:56
1,1.177107,1.209287,0.720000,0.963057,00:56
2,1.068190,1.269524,0.685605,0.957707,00:56
3,1.016310,1.021763,0.801783,0.977834,00:56
4,0.858195,0.870623,0.862166,0.987006,00:56


In [7]:
print(f'Mean of 5 runs: {np.mean([0.857325, 0.865223, 0.868280, 0.865478, 0.862166])}')

Mean of 5 runs: 0.8636944


### With BlurPool

In [5]:
import kornia

# function from https://github.com/ducha-aiki/imagewoofv2-fastv2-maxpoolblur/blob/master/fastai2-imagenette-train-maxblurpool.ipynb
def convert_MP_to_blurMP(model, layer_type_old):
    conversion_count = 0
    for name, module in reversed(model._modules.items()):
        if len(list(module.children())) > 0:
            # recurse
            model._modules[name] = convert_MP_to_blurMP(module, layer_type_old)

        if type(module) == layer_type_old:
            layer_old = module
            layer_new = kornia.contrib.MaxBlurPool2d(3, True)
            model._modules[name] = layer_new

    return model

for i in range(5):
    model = convert_MP_to_blurMP(xse_resnext50(n_out=10, act_cls=Mish, sa=1, sym=0, pool=MaxPool), nn.MaxPool2d)

    learn = Learner(
        get_dataloaders(bs=32, batch_tfms=[], item_tfms=[Resize(size=128), FlipItem(0.5)]),
        model,
        opt_func=partial(ranger, mom=0.95, sqr_mom=0.99, eps=1e-6, beta=0),
        metrics=[accuracy,top_k_accuracy],
        loss_func=LabelSmoothingCrossEntropy()
    )

    learn.fit_flat_cos(5, 8e-3, wd=1e-2, cbs=[])

epoch,train_loss,valid_loss,accuracy,top_k_accuracy,time
0,1.389785,1.317525,0.670828,0.943694,01:11
1,1.180389,1.226909,0.723057,0.965860,01:11
2,1.062296,1.188035,0.715669,0.963822,01:11
3,1.020016,1.042163,0.791847,0.976051,01:11
4,0.831000,0.826279,0.876688,0.987771,01:11


/home/radek/anaconda3/lib/python3.8/site-packages/torch/nn/functional.py:3103: UserWarning: The default behavior for interpolate/upsample with float scale_factor changed in 1.6.0 to align with other frameworks/libraries, and now uses scale_factor directly, instead of relying on the computed output size. If you wish to restore the old behavior, please set recompute_scale_factor=True. See the documentation of nn.Upsample for details. 
  warnings.warn("The default behavior for interpolate/upsample with float scale_factor changed "


epoch,train_loss,valid_loss,accuracy,top_k_accuracy,time
0,1.362602,1.225767,0.703185,0.962293,01:11
1,1.184222,1.167927,0.739873,0.964841,01:11
2,1.073235,1.097123,0.757707,0.973503,01:10
3,0.980343,1.136434,0.751083,0.963057,01:11
4,0.819502,0.831387,0.872611,0.988535,01:11


epoch,train_loss,valid_loss,accuracy,top_k_accuracy,time
0,1.464863,1.566745,0.559236,0.937834,01:11
1,1.221598,1.311862,0.665223,0.956688,01:11
2,1.113531,1.114526,0.760000,0.972994,01:11
3,1.039181,1.077434,0.773758,0.974013,01:11
4,0.852318,0.842802,0.868280,0.986242,01:11


epoch,train_loss,valid_loss,accuracy,top_k_accuracy,time
0,1.385718,1.357973,0.651465,0.946752,01:11
1,1.151854,1.231295,0.715669,0.958726,01:11
2,1.065293,1.041506,0.798726,0.972229,01:11
3,1.003920,0.996093,0.799236,0.978344,01:11
4,0.814105,0.823766,0.874904,0.988790,01:11


epoch,train_loss,valid_loss,accuracy,top_k_accuracy,time
0,1.346775,1.325512,0.665733,0.948280,01:10
1,1.184837,1.190067,0.721783,0.960510,01:11
2,1.053629,1.192228,0.736306,0.967389,01:11
3,0.995223,0.975204,0.810191,0.982930,01:11
4,0.822578,0.824715,0.879236,0.990318,01:10


In [8]:
print(f'Mean of 5 runs: {np.mean([0.876688, 0.872611, 0.868280, 0.874904, 0.879236])}')

Mean of 5 runs: 0.8743437999999999
